In [201]:
#importing python libraries
from pydantic import Field,BaseModel
from typing import Annotated
from IPython.display import display,Image
from dotenv import load_dotenv
import os
import gradio as gr
import sqlite3

In [202]:
#importing langchain,langgraph
from langgraph.graph import StateGraph,START,END,add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import GoogleSerperAPIWrapper
from langgraph.prebuilt import tools_condition,ToolNode
from langchain.agents import Tool
from langgraph.checkpoint.sqlite import SqliteSaver



In [203]:
dbpath = './memory/memory.db'
dbconn = sqlite3.connect(dbpath,check_same_thread=False)
sql_memory = SqliteSaver(dbconn)

In [204]:
gemini_model = google_model = "gemini-2.0-flash"

In [205]:
load_dotenv(override=True)

True

In [206]:
gemini_api_key = os.getenv("GEMINI_API_KEY")


In [207]:
serper = GoogleSerperAPIWrapper()

In [ ]:
class State(BaseModel):
    messages : Annotated[list,add_messages]

In [209]:
search_tool =Tool(
    name="searchTool",
    func=serper.run,
    description="searching"
)

In [210]:
tools = [search_tool]

In [211]:
graph_builder = StateGraph(State)

In [212]:
llm = ChatGoogleGenerativeAI(api_key=gemini_api_key,model=gemini_model)
tools = [search_tool]
llm_as_tool = llm.bind_tools(tools=tools)

In [ ]:
def chat_bot(state:State)->State:
    result= llm_as_tool.invoke(state.messages)
    return State(messages=[result])



In [214]:
graph_builder.add_node("chatbot",chat_bot)
graph_builder.add_node("tools",ToolNode(tools=tools))

In [215]:
graph_builder.add_conditional_edges("chatbot",tools_condition,"tools")
graph_builder.add_edge("tools","chatbot")
graph_builder.add_edge(START,"chatbot")

In [216]:
graph = graph_builder.compile(checkpointer=sql_memory)

In [217]:
try:
    png_bytes = graph.get_graph().draw_mermaid_png()
    display(Image(data=png_bytes))
except Exception as e:
    print("⚠️ Could not render PNG. Showing Mermaid text only.")
    print("Error:", e)


⚠️ Could not render PNG. Showing Mermaid text only.
Error: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 204.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


In [ ]:
config = {"configurable":{"thread_id":"1"}}
def chat(user_input,history):
    result = graph.invoke(State(messages=[{"role":"user","content":user_input}]),config=config)
    return result["messages"][-1].content

In [ ]:
gr.ChatInterface(chat,type="messages").launch()

ValueError: The `type` parameter must be 'messages' or 'tuples', received: message